# Association Analysis
- unsupervised find common patterns in large datasets
- limited need for data prep and feature engineering

Have to use MLxtend from Sebastian Raschka 


### Market Basket
- Market basket analysis is one of the key applications of machine learning in retail
- Find products that are associated  




### Introduction to Association Analysis
- associations are written like this
    - {Diapers} -> {Beer}
- Diaper is the antecedent and Beer is the consequent
- there are different algos:
    - Apriori
    - Eclat
    - FP-Growth


Apriori is the most popular algorithm and is based on the following concepts:

#### Support
- relative frequency that the rule shows up in the data
- proportion of transactions in which an itemset appears.
- generally you want a high support
  
#### Confidence
- how likely item y is purchased when item X is purchased
- proportion of transactions with item X, in which item Y also appears
- problem: item y could just be popular 

#### Lift
- how likely item Y is purchased when item X is purchased controlling for how popular item Y is.
- ratio of observed  support to that expected 

### Process
1. Transform data into the right format/structure
    - each invoice has a row
    - each product has a column, which is 1 hot encoded
2. Generate frequent item sets
    - use `apriori` 
3. Generate rules
    - use `association_rules`
    - calculates the corresponding support, confidence and lift 

In [1]:
# import
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [25]:
df = pd.read_excel('data/Online Retail.xlsx')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


#### Clean up

In [27]:
# remove the spaces in Description
df['Descriptom'] = df['Description'].str.strip()

In [28]:
# drop rows that don't have invoice numbers
df.dropna(axis=0, subset= ['InvoiceNo'], inplace=True)

In [29]:
# remove credit transactions
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

#### Consolidate the items into 1 transaction per row with each product 1 hot encoded
desired result:
- each invoiceNo has a row
- each product has a column, which is 1 hot encoded

In [37]:
basket = df[df['Country'] == 'France'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum()
basket

InvoiceNo  Description                     
536370      SET 2 TEA TOWELS I LOVE LONDON     24
           ALARM CLOCK BAKELIKE GREEN          12
           ALARM CLOCK BAKELIKE PINK           24
           ALARM CLOCK BAKELIKE RED            24
           CHARLOTTE BAG DOLLY GIRL DESIGN     20
                                               ..
581587     CIRCUS PARADE LUNCH BOX             12
           PACK OF 20 SPACEBOY NAPKINS         12
           PLASTERS IN TIN CIRCUS PARADE       12
           PLASTERS IN TIN STRONGMAN           12
           SPACEBOY LUNCH BOX                  12
Name: Quantity, Length: 8372, dtype: int64

In [40]:
basket = basket.unstack()

In [42]:
basket = basket.fillna(0)

In [43]:
basket 

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# one hot encode items
def encode_items(x):
    if x <= 0:
        return 0
    if x > 0:
        return 1

In [47]:
# apply function with applymap
# applymap applies a function to a dataframe to every element

basket_encoded = basket.applymap(encode_items)

In [48]:
basket_encoded

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# drop Postage column
basket_encoded.drop('POSTAGE', inplace=True, axis=1)

#### Generate frequent items sets

In [51]:
# set support to 7%
frequent_itemsets = apriori(basket_encoded, min_support=0.07, use_colnames=True)

### Final step
- generate the rules with their corresponding support, confidence ans lift

In [53]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [60]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
2,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED ),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


### How to interpret

In [58]:
# filter the rules
# higher than 6 in lift
# higher than 80% confidence
rules[(rules['lift'] >= 6) & (rules['confidence'] >= 0.8)] 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
17,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS ),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
19,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
20,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
21,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
22,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS ),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850


> it appears that green and red alarm clocks are purchased together as well as paper cups, napkins ans plates 

### Transforming the data when it is in a different format

In [64]:
df2 = pd.read_csv('data/retail_dataset.csv')

In [65]:
df2.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


In [73]:
items = df2['0'].unique()
items

array(['Bread', 'Cheese', 'Meat', 'Eggs', 'Wine', 'Bagel', 'Pencil',
       'Diaper', 'Milk'], dtype=object)

In [74]:
encoded_vals = []

for index, row in df.iterrows():
    labels = {}
    uncommons = list(set(items) - set(row))
    commons = list(set(items).intersection(row))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]

df2_encoded = pd.DataFrame(encoded_vals)

In [75]:
df2_encoded

,Diaper,Meat,Bagel,Cheese,Eggs,Bread,Wine,Milk,Pencil
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
532616,0,0,0,0,0,0,0,0,0
532617,0,0,0,0,0,0,0,0,0
532618,0,0,0,0,0,0,0,0,0
532619,0,0,0,0,0,0,0,0,0


> now the dataset can be used with the methods from mlextand 

### Resources
- https://pbpython.com/market-basket-analysis.html
- https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html
- https://analyticsindiamag.com/hands-on-guide-to-market-basket-analysis-with-python-codes/
- https://medium.com/analytics-vidhya/association-analysis-in-python-2b955d0180c